In [1]:
#Import packages
import pandas as pd
import numpy as np
import re
import nltk
from scipy.spatial import distance
import pyreadstat
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import string
from sklearn.metrics.pairwise import cosine_distances

In [3]:
# read in data
all_data = pd.read_csv('/Users/ncarlson/Dropbox/Mintel Immigration Project/Code and Analysis/Final Matches/Mintel_Matched.csv')
all_data = all_data.drop_duplicates(subset=['Record.ID'])
all_data = all_data[all_data['Product.Description'].notnull()]
# reset index
all_data = all_data.reset_index()
print(len(all_data))

112880


In [7]:
# create sentence embedding measure

# transformer model
model = SentenceTransformer('all-mpnet-base-v2')

# sentence embeddings
all_data = all_data[all_data['Product.Description'].notnull()]
sentences = list(all_data['Product.Description'])
#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

In [8]:
np.save("embeddings", embeddings)

In [4]:
embeddings = np.load("/Users/ncarlson/Dropbox/Mintel Immigration Project/Code and Analysis/Final Matches/embeddings.npy")

In [5]:
print(len(all_data))
all_data = all_data.drop_duplicates(subset=['Record.ID'])
print(len(all_data))
#print(all_data.head())
index = all_data.index
print(len(embeddings))
embeddings = np.take(embeddings,index,axis=0)
print(len(embeddings))
# reset index
all_data = all_data.reset_index()
#embeddings = embeddings.reset_index()

112880
112880
112880
112880


In [18]:
# transformer model
model = SentenceTransformer('all-mpnet-base-v2')
# create axis for each country in list
# extract unique country names
unique_countries = all_data['Market'].unique().tolist()

# Encode "United States"
us_embedding = model.encode("United States")

# Create a dictionary to store the embeddings
country_embeddings_dict = {}

for country in unique_countries:
    country_embedding = model.encode(country)
    diff_embedding = us_embedding - country_embedding
    country_embeddings_dict[country] = diff_embedding

# Display the result
#country_embeddings_dict

In [11]:
def cosine_distance(emb1, emb2):
    return cosine_distances([emb1], [emb2])[0][0]

# Add a new column "Localization" to the dataframe
localization = []

for i, row in all_data.iterrows():
    market = row['Market']
    if market == 'USA':
        localization.append(1)
    else:
        product_embedding = embeddings[i]
        country_embedding = country_embeddings_dict.get(market)
        if country_embedding is not None:
            distance = cosine_distance(product_embedding, country_embedding)
            localization.append(distance)
        else:
            localization.append(None)  # Handle cases where the country embedding is not found

all_data['Localization'] = localization

In [15]:
# Define the countries of interest
countries_of_interest = ['China', 'India', 'Mexico', 'Canada', 'Nigeria', 'Japan', 'UK', 'South Korea', 'Argentina']

# Function to print the top 5 highest and lowest product descriptions for a given country
def print_top_and_bottom_descriptions(country, df):
    country_df = df[df['Market'] == country]
    if not country_df.empty:
        top_descriptions = country_df.nlargest(5, 'Localization')['Product.Description']
        bottom_descriptions = country_df.nsmallest(5, 'Localization')['Product.Description']
        
        print(f"Top 5 highest localization scores for {country}:")
        for i, description in enumerate(top_descriptions, 1):
            print(f"{i}. {description}")
        print()
        
        print(f"Top 5 lowest localization scores for {country}:")
        for i, description in enumerate(bottom_descriptions, 1):
            print(f"{i}. {description}")
        print()
    else:
        print(f"No data available for {country}")

# Iterate over the countries of interest and print the top and bottom descriptions
for country in countries_of_interest:
    print_top_and_bottom_descriptions(country, all_data)

Top 5 highest localization scores for China:
1. Member's Mark Bai Cai Zhu Rou Shui Jiao (Chinese Cabbage and Pork Dumplings) are made using fresh pork, which has an equal proportion of fat and lean meat, and selected Northern Chinese cabbages, which are said to be sweet and tasty. Each dumpling has 50% filling content, and is said to be fleshy, fresh, juicy, chewy and smooth. This product is processed according to vacuum knead dough technique, and retails in a 1.2kg pack containing three 400g easy to store individual units and bearing a QR code.
2. Wanchai Ferry Qin Cai Lian Ou Zhu Rou Shou Gong Shui Jiao (Hand-Made Dumplings with Celery, Lotus Root, and Pork) are made with selected top grade celery, lotus root and pork. Each one is wrapped with a thin pastry and filled with a generous and juicy filling. This product retails in a 1kg pack containing 48 units.
3. Wanchai Ferry Jiang Nan Xian Shanghai Ji Cai Zhu Rou Da Hun Tun (Shanghai Style Shepherd's-Purse and Pork Wonton) is made wit

In [17]:
# write to csv
all_data.to_csv('/Users/ncarlson/Dropbox/Mintel Immigration Project/Code and Analysis/Final Matches/Mintel_Localization.csv')